# Reproduction for CGO-24 submition

In [1]:
import os, sys, json

autocog_home = os.getenv('AUTOCOG_HOME', '/workspace/projects/autocog')
sys.path.append(autocog_home)

from autocog import CogArch
from autocog.lm import OpenAI, TfLM, Llama
from autocog.architecture.utility import PromptTee

## Load program

Prompts will be echoed to `stdout` and text files in the `results/cgo24` directory.
FIXME: all executions of each prompt (within one frame) are append to `name-0.txt` instead of `name-0-0.txt`, `name-1-0.txt`, `name-2-0.txt`, ... We are missing the sequence id (visible 0 is mapping-id).

`arch.load` can receive `kwargs` which are used for macros (`f-exp` substitution in the program and `?` statement provide default).

In [2]:
!rm -rf results
arch = CogArch(pipe=PromptTee(prefix='cgo24', tee=sys.stdout, fmt='results/{p}/{c}/{t}-{i}.txt'))
_ = arch.load(tag='cgo24', filepath='program.sta', T=3, N=2, R=2, S=3, L=2)

In [3]:
openai_model = 'text-davinci-003'
#openai_model = 'text-curie-001'
arch.orchestrator.LMs.update({
  'text'     : OpenAI(model=openai_model, max_tokens=20, temperature=0.4),
  'thought'  : OpenAI(model=openai_model, max_tokens=15, temperature=1.0),
  'sentence' : OpenAI(model=openai_model, max_tokens=50, temperature=0.7),
})
res = await arch('cgo24', question="Explain the different phases of a compiler")
print(json.dumps(res[0], indent=4))



 === cgo24[0] === 

You are a helpful AI assistant.
Given a user question, you craft an answer
formulate your initial answer
You are using an interactive questionnaire.
Follow this structure after the start prompt:
```
> question(text): user's question
> ponder[3](thought): you think about an answer
> answer[2](sentence): your initial answer
```
Each prompt expects one of the following formats:
- text: ASCII text in any form
- thought: your thoughts (a few words per lines)
- sentence: one natural language sentence per line
Terminate each prompt with a newline. Use as many statement with `thought` format as needed.

start(record):
> question(text): Explain the different phases of a compiler
> ponder[1](thought):  Compiling program source code into machine code.  
> ponder[2](thought):  Tokenizing, syntax analysis, semantic analysis, optional code optimization.  
> answer[1](sentence):  Compiling a program involves various steps, such as tokenizing, syntax analysis, semantic analysis, 

In [4]:
llama_path = lambda x: "/workspace/models/{}/ggml-model-{}.bin".format(*x)
model_kwargs = Llama.create(model_path=llama_path(('7B','q4_0')), n_ctx=2048)
arch.orchestrator.LMs.update({
  'text'     : Llama(**model_kwargs, completion_kwargs={ 'max_tokens' : 20, 'temperature' : 0.4 }),
  'thought'  : Llama(**model_kwargs, completion_kwargs={ 'max_tokens' : 15, 'temperature' : 1.0 }),
  'sentence' : Llama(**model_kwargs, completion_kwargs={ 'max_tokens' : 50, 'temperature' : 0.7 }),
}) # llama-cpp-python defaults: top_p=0.95, top_k=40, repeat_penalty=1.1
res = await arch('cgo24', question="Explain the different phases of a compiler")
print(json.dumps(res[0], indent=4))

llama.cpp: loading model from /workspace/models/7B/ggml-model-q4_0.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =  68.20 KB
llama_model_load_internal: mem required  = 5809.33 MB (+ 1026.00 MB per state)




 === cgo24[1] === 

You are a helpful AI assistant.
Given a user question, you craft an answer
formulate your initial answer
You are using an interactive questionnaire.
Follow this structure after the start prompt:
```
> question(text): user's question
> ponder[3](thought): you think about an answer
> answer[2](sentence): your initial answer
```
Each prompt expects one of the following formats:
- text: ASCII text in any form
- thought: your thoughts (a few words per lines)
- sentence: one natural language sentence per line
Terminate each prompt with a newline. Use as many statement with `thought` format as needed.

start(record):
> question(text): Explain the different phases of a compiler
> ponder[1](thought): 

llama_init_from_file: kv self size  = 1024.00 MB


2000 years ago, compilers were more like editors and
> ponder[2](thought): 100 years ago, compilers were written as series of small mac
> ponder[3](thought): 50 years ago, compilers became an AI program by implementing algorithms
> answer[1](sentence): 1964, the first compiler for a programming language was invented.
> answer[2](sentence): 1987-1993, it was written in Pascal and C
You are a helpful AI assistant.
Given a user question, you craft an answer
improve your answer iteratively
You are using an interactive questionnaire.
Follow this structure after the start prompt:
```
> question(text): user's question
> draft[2](sentence): your current answer
> problems[2](record): list all issues in this answer
> > identify(sentence): one issue to address
> > consider[3](thought): solutions for that issue
> answer[2](sentence): write your corrected answer
> issues(thought): are there other issues left to edit?
```
Each prompt expects one of the following formats:
- next: "edit" the issues or

In [5]:
from autocog.utility.pynb import wrap_graphviz
wrap_graphviz(arch.toGraphViz())